## Modelo

$$
\begin{align}
    \text{max} \quad & \sum_i{z_i V_i M_i - C_i P_i} \\
    \text{s.t.} \quad & \sum_i{C_i} \leq R_{max} \sum_i{z_i V_i M_i - C_i P_i} \\
    & R_i \geq r_{i} - M(1-\delta_i) \quad \forall ~i \\
    & R_i \leq r_{i} - \epsilon + M \delta_i \quad \forall ~i \\
    & 1 - M (1-\delta_i) <= z_i <= 1 + M (1-\delta_i) \quad \forall ~i \\
    & - M \delta_i <= z_i <= M \delta_i \quad \forall ~i \\

    & z_i \geq \epsilon - M(1-\alpha_i) \quad \forall ~i \\
    & z_i \leq M \alpha_i \quad \forall ~i \\
    & R_i V_i - M (1-\alpha_i) <= C_i <= R_i V_i + M (1-\alpha_i) \quad \forall ~i \\
    & - M \alpha_i <= C_i <= M \alpha_i \quad \forall ~i \\
    
    & z_{i},\alpha_{i},\delta_{i}   \in \left \{ 0, 1 \right \} \forall ~i \in I\\
    & R_i >= 0
\end{align}
$$

In [11]:
import pandas as pd
import pyomo.environ as pyo
from pyomo.contrib.appsi.solvers.highs import Highs

In [12]:
# Carrega o dataframe
df_risco = pd.read_excel('risco.xlsx')

In [13]:
# Cria o modelo
model = pyo.ConcreteModel()

In [14]:
# Índices
model.I = pyo.Set(initialize=df_risco.index)

# Parâmetros
model.Prob_inad = pyo.Param(model.I, initialize={idx: val for idx, val in df_risco['Prob_inad'].items()})
model.VLRIMV = pyo.Param(model.I, initialize={idx: val for idx, val in df_risco['VLRIMV'].items()})
model.Margem = pyo.Param(model.I, initialize={idx: val for idx, val in df_risco['Margem'].items()})
model.Risco_min_compra = pyo.Param(model.I, initialize={idx: val for idx, val in df_risco['Risco_min_compra'].items()})
model.eps = pyo.Param(initialize = 0.000000001)
model.big_M = pyo.Param(initialize = 100000000)
model.risco_max = pyo.Param(initialize = 0.2)
model.min_compra = pyo.Param(initialize = 0.2)

In [15]:
# Variáveis de decisão
model.Risco = pyo.Var(model.I, within=pyo.NonNegativeReals)
model.Financiamento = pyo.Var(model.I, within=pyo.NonNegativeReals)
model.Compra = pyo.Var(model.I, within=pyo.Binary)
model.Aux_compra = pyo.Var(model.I, within=pyo.Binary)
model.Aux_fnc = pyo.Var(model.I, within=pyo.Binary)


In [16]:
# Função objetivo
# Lucro_esperado = VLRIMV * Margem_ant * Decisao_compra - Valor_financiado * Probabilidade_inadimplencia
def lucro_func(model):
    return sum(model.Aux_compra[idx] * model.VLRIMV[idx] * model.Margem[idx] - model.Financiamento[idx] * model.Prob_inad[idx] for idx in model.I)

model.obj = pyo.Objective(rule=lucro_func, sense=maximize)

In [17]:
# Restrições

# Carteira <= risco_max * Lucro 
def financiamento_max_func(model):
    return sum(model.Financiamento[idx] for idx in model.I) <= model.risco_max * model.obj

# Quantidade mínima de compras
def min_compra_func(model):
    return sum(model.Compra[idx] for idx in model.I) >= model.min_compra * len(model.I)

# if risco >= risco_min então compra = 1 else compra = 0
def risco_min_compra_func(model, idx):
    return model.Risco[idx] >= model.Risco_min_compra[idx] - model.big_M * (1 - model.Aux_compra[idx])

def risco_min_compra_func2(model, idx):
    return model.Risco[idx] <= model.Risco_min_compra[idx] - model.eps + model.big_M * model.Aux_compra[idx]

def compra_constraints_1(model, idx):
    return model.Compra[idx] >= 1 - model.big_M * (1 - model.Aux_compra[idx])

def compra_constraints_2(model, idx):
    return model.Compra[idx] <= 1 + model.big_M * (1 - model.Aux_compra[idx])

def compra_constraints_3(model, idx):
    return model.Compra[idx] >= - model.big_M * model.Aux_compra[idx]

def compra_constraints_4(model, idx):
    return model.Compra[idx] <= model.big_M * model.Aux_compra[idx]


# if compra > 0 então financiamento = vlr_imv*risco else financiamento = 0
def fnm_compra_constraint_1(model, idx):
    return model.Compra[idx] >= 0 + model.eps - model.big_M * (1 - model.Aux_fnc[idx])

def fnm_compra_constraint_2(model, idx):
    return model.Compra[idx] <= 0 + model.big_M *model.Aux_fnc[idx]

def financiamento_constraints_1(model, idx):
    return model.Financiamento[idx] >= model.Risco[idx] * model.VLRIMV[idx] - model.big_M * (1 - model.Aux_fnc[idx])

def financiamento_constraints_2(model, idx):
    return model.Financiamento[idx] <= model.Risco[idx] * model.VLRIMV[idx] + model.big_M * (1 - model.Aux_fnc[idx])

def financiamento_constraints_3(model, idx):
    return model.Financiamento[idx] >= -model.Aux_fnc[idx] * model.big_M

def financiamento_constraints_4(model, idx):
    return model.Financiamento[idx] <= model.Aux_fnc[idx] * model.big_M


model.financiamento_max = pyo.Constraint(rule=financiamento_max_func)
model.min_compra_ = pyo.Constraint(rule=min_compra_func)

model.risco_min_compra = pyo.Constraint(model.I, rule=risco_min_compra_func)
model.risco_min_compra2 = pyo.Constraint(model.I, rule=risco_min_compra_func2)
model.compra_constraints_1 = pyo.Constraint(model.I, rule=compra_constraints_1)
model.compra_constraints_2 = pyo.Constraint(model.I, rule=compra_constraints_2)
model.compra_constraints_3 = pyo.Constraint(model.I, rule=compra_constraints_3)
model.compra_constraints_4 = pyo.Constraint(model.I, rule=compra_constraints_4)

model.fnm_compra_constraint_1 = pyo.Constraint(model.I, rule=fnm_compra_constraint_1)
model.fnm_compra_constraint_2 = pyo.Constraint(model.I, rule=fnm_compra_constraint_2)
model.financiamento_constraints_1 = pyo.Constraint(model.I, rule=financiamento_constraints_1)
model.financiamento_constraints_2 = pyo.Constraint(model.I, rule=financiamento_constraints_2)
model.financiamento_constraints_3 = pyo.Constraint(model.I, rule=financiamento_constraints_3)
model.financiamento_constraints_4 = pyo.Constraint(model.I, rule=financiamento_constraints_4)


In [18]:
# Resolve o modelo
solver = Highs()
results = solver.solve(model)

In [19]:
print("Lucro Esperado: R$", round(model.obj(),2))
cart = sum(model.Financiamento[idx].value for idx in model.I)
risco_tot = 100*cart/model.obj()
print(f"Carteira:{cart} \nRisco: {round(risco_tot,1)}%")

Lucro Esperado: R$ 1219970.13
Carteira:242410.06594204903 
Risco: 19.9%


In [20]:
model.Risco.display()

Risco : Size=18, Index=I
    Key : Lower : Value                : Upper : Fixed : Stale : Domain
      0 :     0 :  0.01000000536441803 :  None : False : False : NonNegativeReals
      1 :     0 : 0.019999995827674866 :  None : False : False : NonNegativeReals
      2 :     0 : 0.030000001192092896 :  None : False : False : NonNegativeReals
      3 :     0 :                 -0.0 :  None : False : False : NonNegativeReals
      4 :     0 :  0.04999999701976776 :  None : False : False : NonNegativeReals
      5 :     0 :  0.15000000596046448 :  None : False : False : NonNegativeReals
      6 :     0 :  0.04999999701976776 :  None : False : False : NonNegativeReals
      7 :     0 :  0.07999999821186066 :  None : False : False : NonNegativeReals
      8 :     0 :                  0.0 :  None : False : False : NonNegativeReals
      9 :     0 :  0.10999999940395355 :  None : False : False : NonNegativeReals
     10 :     0 :                  0.0 :  None : False : False : NonNegativeReals
 

In [21]:
df_risco['Risco'] = [model.Risco[idx].value for idx in model.I]
df_risco['Carteira'] = [model.Financiamento[idx].value for idx in model.I]
df_risco['Compra'] = [model.Compra[idx].value for idx in model.I]
df_risco

,Cliente,VLRIMV,Margem,Prob_inad,Risco_min_compra,Risco,Carteira,Compra
0,1,254144,0.37,0.44,0.01,0.01,2541.441363,1.0
1,2,287788,0.34,0.12,0.02,0.02,5755.758799,1.0
2,3,255676,0.35,0.17,0.03,0.03,7670.280305,1.0
3,4,224552,0.40,0.11,0.18,-0.00,0.000000,0.0
4,5,221604,0.34,0.30,0.05,0.05,11080.199340,1.0
5,6,265227,0.40,0.11,0.15,0.15,39784.051581,1.0
6,7,251343,0.34,0.11,0.05,0.05,12567.149251,1.0
7,8,236892,0.34,0.38,0.08,0.08,18951.359576,1.0
8,9,190423,0.34,0.23,0.19,0.00,0.000000,0.0
9,10,241605,0.34,0.28,0.11,0.11,26576.549856,1.0
